# 1-Keras入门
## 笑脸识别

In [4]:
import numpy as np
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
import kt_utils
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

### (1)加载数据集

In [3]:
X_train_orig,Y_train_orig,X_test_orig,Y_test_orig,classes = kt_utils.load_dataset()

X_train = X_train_orig / 255
X_test = X_test_orig / 255
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((600, 64, 64, 3), (600, 1), (150, 64, 64, 3), (150, 1))

### (2)使用Keras框架搭建模型

In [7]:
def HappyModel(input_shape):
    """笑脸检测模型"""
    # 定义一个placeholder维度为input_shape
    X_input = Input(input_shape)
    
    # 使用0填充X_input
    X = ZeroPadding2D((3,3))(X_input)
    
    # 对X使用Conv->BN->Relu块
    X = Conv2D(32,(7,7),strides=(1,1),name='conv0')(X)
    X = BatchNormalization(axis=3,name='bn0')(X)
    X = Activation('relu')(X)
    
    # 最大池化层
    X = MaxPooling2D((2,2),name='max_pool')(X)
    
    # 降维成向量并添加全连接层
    X = Flatten()(X)
    X = Dense(1,activation='sigmoid',name='fc')(X)
    
    # 创建模型
    model = Model(inputs=X_input,outputs=X,name='HappyModel')
    
    return model 

### (3)训练并测试模型
### 现在我们已经设计好了我们的模型了，要训练并测试模型我们需要这么做：
### 1.创建一个模型实体。
### 2.编译模型，可以使用这个语句：model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])。
### 3.训练模型：model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)。
### 4.评估模型：model.evaluate(x = ..., y = ...)。

In [9]:
# 创建模型实体
happy_model = HappyModel(X_train.shape[1:])
# 编译模型
happy_model.compile("adam","binary_crossentropy",metrics=['accuracy'])
# 训练模型
happy_model.fit(X_train,Y_train,epochs=40,batch_size=50)
# 评估模型
preds = happy_model.evaluate(X_test,Y_test,batch_size=32,verbose=1,sample_weight=None)

print("误差为：%f,准确度为：%f" % (preds[0],preds[1]))

Epoch 1/40
12/12 [==============================] - 2s 161ms/step - loss: 2.0541 - accuracy: 0.5833
Epoch 2/40
12/12 [==============================] - 2s 159ms/step - loss: 0.4702 - accuracy: 0.8167
Epoch 3/40
12/12 [==============================] - 2s 162ms/step - loss: 0.2138 - accuracy: 0.9033
Epoch 4/40
12/12 [==============================] - 2s 186ms/step - loss: 0.1859 - accuracy: 0.9400
Epoch 5/40
12/12 [==============================] - 2s 166ms/step - loss: 0.1530 - accuracy: 0.9433
Epoch 6/40
12/12 [==============================] - 2s 164ms/step - loss: 0.1091 - accuracy: 0.9650
Epoch 7/40
12/12 [==============================] - 2s 166ms/step - loss: 0.1007 - accuracy: 0.9750
Epoch 8/40
12/12 [==============================] - 2s 166ms/step - loss: 0.1088 - accuracy: 0.9600
Epoch 9/40
12/12 [==============================] - 2s 164ms/step - loss: 0.1206 - accuracy: 0.9567
Epoch 10/40
12/12 [==============================] - 2s 166ms/step - loss: 0.0897 - accuracy: 0.9750

### (4) 打印模型每层细节

In [10]:
happy_model.summary()

Model: "HappyModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 zero_padding2d_2 (ZeroPaddi  (None, 70, 70, 3)        0         
 ng2D)                                                           
                                                                 
 conv0 (Conv2D)              (None, 64, 64, 32)        4736      
                                                                 
 bn0 (BatchNormalization)    (None, 64, 64, 32)        128       
                                                                 
 activation_2 (Activation)   (None, 64, 64, 32)        0         
                                                                 
 max_pool (MaxPooling2D)     (None, 32, 32, 32)        0         
                                                        